## CM3 data로 복습하기 1

In [1]:
# CM3 data로 복습하기 1 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 
import sweetviz as sv
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings("ignore")


c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 분석 연습 파일 불러오기
cm3 = pd.read_excel("d:/novelis/ds2023/CM3_분석연습.xls")
cm3.head(2)

,구분,회수율,시작일시,종료일시,준비시간,작업시간,소요시간(UPTIME),Batch No,YS_TARGET_온도,Make as Alloy,...,GAUGE 3SIGMA,FLATNESS 3SIGMA,ENTRY WEIGHT,2STD 평균속도,2STD 최대속도,2STD BENDING,2STD ROLL FORCE,입측 COIL온도,평균 출측 온도,COIL TEMP DIFF
0,NaN,97.21,2023-01-01 06:27:25,2023-01-01 06:37:00,2.83,6.75,9.58,S843331,165.0,M3104,...,0.82,2.76,11881.0,1109.0,1392.0,14.3,489.0,58.0,142.53360,-22.46640
1,NaN,97.30,2023-01-01 06:37:00,2023-01-01 06:46:01,2.73,6.28,9.01,S843341,165.0,M3104,...,0.78,3.12,11876.0,1180.0,1518.0,16.7,485.0,58.0,142.68825,-22.31175


In [5]:
cm3.shape # 행의 개수와 열의 개수 

(30931, 36)

In [6]:
cm3.columns # 열의 이름

Index(['구분', '회수율', '시작일시', '종료일시', '준비시간', '작업시간', '소요시간(UPTIME)', 'Batch No',
       'YS_TARGET_온도', 'Make as Alloy', 'Pass', 'ENTRY중량', 'EXIT중량', '작업자',
       '폭', '후공정', '입측텐션', '인터텐션', '출측텐션', '판검사', '회수율내역', '일연실적일', 'TR속도',
       'ENTRY GAUGE', 'INTER GAUGE', 'EXIT GAUGE', 'GAUGE 3SIGMA',
       'FLATNESS 3SIGMA', 'ENTRY WEIGHT', '2STD 평균속도', '2STD 최대속도',
       '2STD BENDING', '2STD ROLL FORCE', '입측 COIL온도', '평균 출측 온도',
       'COIL TEMP DIFF'],
      dtype='object')

In [10]:
cm3_a = cm3[['구분','Batch No', '회수율', '시작일시', '종료일시', '준비시간', '작업시간', '소요시간(UPTIME)']]
cm3_b = cm3[['Batch No','시작일시', '종료일시','YS_TARGET_온도', 'Make as Alloy', 'Pass','폭', 'ENTRY GAUGE', 'INTER GAUGE', 'EXIT GAUGE']]
cm3_c = cm3[['Batch No','시작일시', '종료일시','2STD BENDING','2STD 평균속도', '2STD 최대속도', '2STD ROLL FORCE', '입측 COIL온도', '평균 출측 온도']]

In [13]:
cm3_a.dtypes 

구분                      object
Batch No                object
회수율                    float64
시작일시            datetime64[ns]
종료일시            datetime64[ns]
준비시간                   float64
작업시간                   float64
소요시간(UPTIME)           float64
dtype: object

In [83]:
import datetime as dt 

# %Y: 연도를 4자리로 나타내고 싶을 때 사용함
# %m: 월을 2자리로 나타내고 싶을 때 사용함
# %d: 일을 2자리로 나타내고 싶을 때 사용함 

# "{:%Y-%m-%d %H:%M:%S}".format(data)

In [84]:
# datatime64 타입으로 변경
# cm3_a["시작일시"] = pd.to_datetime(cm3_a["시작일시"], format ="%Y-%m-%d", errors = 'raise')

In [85]:
# 각 열의 타입 확인
cm3_a.dtypes 

구분                      object
Batch No                object
회수율                    float64
종료일시            datetime64[ns]
준비시간                   float64
작업시간                   float64
소요시간(UPTIME)           float64
DATE                    object
month                    int32
day                      int32
time                    object
dtype: object

In [86]:
# cm3_a["DATE"]  = cm3_a["시작일시"].dt.date   # YYYY-MM-DD(문자)
# cm3_a["month"] = cm3_a["시작일시"].dt.month  # 월(숫자)
# cm3_a["day"]   = cm3_a["시작일시"].dt.day    # 일(숫자)
# cm3_a["time"]  = cm3_a["시작일시"].dt.time   # 시(숫자)


In [87]:
# 에러 발생
# A조: 06:30 ~ 14:30 / B조: 14:30 ~ 22:30 / C조: 22:30 ~ 06:30
# cm3_a["조"] = cm3["시작일시"].apply(lambda x: "%d"-1 if x == '%M'<6, '%M'<30  else "%d")


In [88]:
# cm3_a.set_index("시작일시",inplace = True)

In [89]:
cm3_a.head()

,구분,Batch No,회수율,종료일시,준비시간,작업시간,소요시간(UPTIME),DATE,month,day,time
시작일시,,,,,,,,,,,
2022-12-31 17:27:25,NaN,S843331,97.21,2023-01-01 06:37:00,2.83,6.75,9.58,2023-01-01,1,1,06:27:25
2022-12-31 17:37:00,NaN,S843341,97.30,2023-01-01 06:46:01,2.73,6.28,9.01,2023-01-01,1,1,06:37:00
2022-12-31 17:46:00,NaN,S841842,97.47,2023-01-01 06:56:10,2.87,7.28,10.15,2023-01-01,1,1,06:46:00
2022-12-31 17:56:10,NaN,S841912,95.40,2023-01-01 07:05:21,2.73,6.45,9.18,2023-01-01,1,1,06:56:10
2022-12-31 18:05:21,NaN,S841911,96.90,2023-01-01 07:14:58,2.70,6.92,9.62,2023-01-01,1,1,07:05:21


In [90]:
# cm3_a.index = cm3_a.index + pd.Timedelta(hours = 6, minutes=30)

In [91]:
cm3_소요시간 = cm3_a.resample('D')['소요시간(UPTIME)'].mean().round(2)

In [122]:
# sns.barplot(x = cm3_소요시간.index, y = cm3_소요시간.values)
# plt.show()

In [93]:
cm3_d = cm3[['Batch No','시작일시', '평균 출측 온도']]

In [94]:
cm3_d.shape

(30931, 3)

In [95]:
cm3_d.set_index("시작일시",inplace = True)

In [96]:
cm3_d.head(10)

,Batch No,평균 출측 온도
시작일시,,
2023-01-01 06:27:25,S843331,142.53360
2023-01-01 06:37:00,S843341,142.68825
2023-01-01 06:46:00,S841842,142.10706
2023-01-01 06:56:10,S841912,146.37471
2023-01-01 07:05:21,S841911,147.28151
2023-01-01 07:14:58,S841913,147.57454
2023-01-01 07:24:26,S841914,146.67485
2023-01-01 07:33:43,S841891,147.22890
2023-01-01 07:43:21,S841905,145.78455


In [97]:
# cm3_d.index = cm3_a.index + pd.Timedelta(hours = 6, minutes=30)

In [114]:
cm3_d['day'] = cm3_d.index.day
cm3_d.head()


,Batch No,평균 출측 온도,day
시작일시,,,
2023-01-01 06:27:25,S843331,147.67,1
2023-01-01 06:37:00,S843341,147.67,1
2023-01-01 06:46:00,S841842,147.67,1
2023-01-01 06:56:10,S841912,147.67,1
2023-01-01 07:05:21,S841911,147.67,1


In [119]:
cm3_d['avg. exit temp'] = cm3_d['평균 출측 온도'].mean().round(2)
cm3_d

,Batch No,평균 출측 온도,day,avg. exit temp
시작일시,,,,
2023-01-01 06:27:25,S843331,147.67,1,147.67
2023-01-01 06:37:00,S843341,147.67,1,147.67
2023-01-01 06:46:00,S841842,147.67,1,147.67
2023-01-01 06:56:10,S841912,147.67,1,147.67
2023-01-01 07:05:21,S841911,147.67,1,147.67
...,...,...,...,...
2023-08-25 08:44:32,NaN,147.67,25,147.67
2023-08-25 08:51:00,NaN,147.67,25,147.67
2023-08-25 13:47:00,NaN,147.67,25,147.67


In [120]:
# 하나의 열 삭제하기 
del cm3_d["평균 출측 온도"]
cm3_d

,Batch No,day,avg. exit temp
시작일시,,,
2023-01-01 06:27:25,S843331,1,147.67
2023-01-01 06:37:00,S843341,1,147.67
2023-01-01 06:46:00,S841842,1,147.67
2023-01-01 06:56:10,S841912,1,147.67
2023-01-01 07:05:21,S841911,1,147.67
...,...,...,...
2023-08-25 08:44:32,NaN,25,147.67
2023-08-25 08:51:00,NaN,25,147.67
2023-08-25 13:47:00,NaN,25,147.67


In [121]:
# Trend graph 그리기

# sns.barplot(x = cm3_d["day"], 
#             y = cm3_d["평균 출측 온도"])
# plt.title('Avg. Exit temperature of CM3')
# plt.xlabel('Date', fontsize = 10)
# plt.ylabel('Avg. Exit temperature of strip', fontsize=10)
# plt.xticks(rotation = 80, fontsize = 10)
# sns.color_palette("flare")
# plt.show()